# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
%pip install langchain

In [ ]:
%pip install langchain-openai

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [1]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [2]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [4]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.2)


In [5]:
prompt = ChatPromptTemplate.from_template(  "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

In [6]:
# old code 
#chain = LLMChain(llm=llm, prompt=prompt)
#correct code
chain = prompt | llm


In [8]:
product = "Rubber decks"
chain.invoke(product)

AIMessage(content='Introducing Robber Dicks - the perfect tool for all your thieving needs! With their discreet design and adjustable length, these dicks are sure to help you snatch and grab with ease. Say goodbye to clumsy hands and hello to smooth, efficient stealing. Get your hands on Robber Dicks today and start swiping in style!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 28, 'total_tokens': 95, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e70b8d5b-968d-4505-b29d-df6432eef2e8-0', usage_metadata={'input_tokens': 28, 'output_tokens': 67, 'total_tokens': 95, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## SimpleSequentialChain

In [9]:
from langchain.chains import SimpleSequentialChain

In [10]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
"Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

C:\Users\diego\AppData\Local\Temp\ipykernel_43164\1498939052.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [34]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review {review}, rate the quality of this product from zero to ten."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [35]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Introducing the ultimate bath time companion - the Rubber Duck! This adorable little quacker is guaranteed to bring a smile to your face as you soak and relax in the tub. With its waterproof design and durable construction, this duck is built to last through many bubbly adventures. So say goodbye to boring baths and add some quack-tion to your routine with the Rubber Duck!
I would rate the quality of the Rubber Duck as a 9 out of 10. It is durable, waterproof, and guaranteed to bring joy to your bath time experience.

> Finished chain.


{'input': 'Rubber ducks',
 'output': 'I would rate the quality of the Rubber Duck as a 9 out of 10. It is durable, waterproof, and guaranteed to bring joy to your bath time experience.'}

**Repeat the above twice for different products**

## SequentialChain

In [55]:
from langchain.chains import SequentialChain

In [56]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into french. "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='fr_review' #Give a name to your output
                    )


In [57]:
second_prompt = ChatPromptTemplate.from_template(
    'Based on {fr_review}, summarize this review in less than 10 words in french.'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='fr_short_review' #give a name to this output
                    )


In [58]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {fr_short_review} into english"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='en_short_review'
                      )


In [59]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"Rate the quality of the translation of {fr_short_review} into {en_short_review} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='rating'
                     )


In [62]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=['fr_short_review','en_short_review', 'rating'],
    verbose=True
)

In [64]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'fr_short_review': 'Goût moyen, mousse qui ne tient pas, peut-être contrefaçon.',
 'en_short_review': 'Average taste, foam that does not hold, may be counterfeit.',
 'rating': 'I would rate the quality of the translation a 9 out of 10. It accurately captures the meaning of the original phrase and conveys it clearly in English. The only potential improvement could be in finding a more precise translation for "Goût moyen" which could convey the idea of average taste more effectively.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [66]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [79]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}


In [80]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [81]:
llm = ChatOpenAI(temperature=0)

In [82]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [83]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [84]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [76]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content)

chain = (
    RunnablePassthrough()
    | {
        "input": RunnablePassthrough(),
        "chosen_prompt": router_chain,
    }
    | RunnableLambda(lambda x: prompt_templates[x["chosen_prompt"]].format(input=x["input"]))
    | llm
)



In [85]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")


In [86]:
response

{'input': 'Explain what principle component analysis is using linear algebra',
 'text': 'Principal Component Analysis (PCA) is a technique used in machine learning and data analysis to reduce the dimensionality of a dataset while preserving as much variance as possible. \n\nIn linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset. The eigenvectors represent the directions of maximum variance in the data, while the eigenvalues represent the amount of variance along each eigenvector.\n\nTo perform PCA, we first center the data by subtracting the mean of each feature from the data points. Then, we calculate the covariance matrix of the centered data. Next, we find the eigenvectors and eigenvalues of the covariance matrix. \n\nThe eigenvectors form a new basis for the data, where the first eigenvector corresponds to the direction of maximum variance, the second eigenvector corresponds to the direction of second maximum variance,

In [87]:
chain.invoke("What is black body radiation?")

{'input': 'What is black body radiation?',
 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. A black body absorbs all radiation that falls on it, with none being reflected or transmitted. \n\nThe radiation emitted by a black body is characterized by its intensity and spectral distribution, which is determined solely by its temperature. As the temperature of a black body increases, the intensity of the radiation it emits also increases, and the peak wavelength of the radiation shifts to shorter wavelengths.\n\nThis phenomenon is described by Planck's law of black body radiation, which states that the spectral radiance of a black body at a given wavelength and temperature is proportional to the frequency of the radiation and follows a specific distribution known as the Planck distribution.\n\nBlack body radiation plays a crucial role in various fi

In [89]:
chain.invoke("what is 2 + 2")

{'input': 'what is 2 + 2', 'text': 'The answer to 2 + 2 is 4.'}

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**